<a href="https://colab.research.google.com/github/isaac-altair/SQL/blob/master/Various_SQL_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install duckdb

In [2]:
import duckdb
import pandas as pd

In [9]:
df = pd.read_excel("/content/dummy_file.xlsx")

In [ ]:
con = duckdb.connect()

In [13]:
duckdb.query("SELECT * FROM df").df()

,ID,Name,Score
0,1,Alice,85
1,2,Bob,92
2,3,Charlie,78


In [14]:
#####

1. Select the top 3 departments with at least ten employees and highest average salary

In [132]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Departments
departments = pd.DataFrame({
    "dept_id": [10,20,30,40,50],
    "dept_name": ["Engineering","Sales","Data Science","HR","Marketing"]
})

# Employees
num_employees = 100
emp_ids = list(range(1, num_employees+1))
emp_names = [f"Emp{i}" for i in emp_ids]

# Assign departments randomly, with some departments having more employees
dept_ids = np.random.choice([10,20,30,40,50], size=num_employees, p=[0.3,0.25,0.2,0.15,0.1])

# Random salaries by dept_id ranges
salaries = []
for d in dept_ids:
    if d == 10:  # Engineering
        salaries.append(np.random.randint(75000,120000))
    elif d == 20:  # Sales
        salaries.append(np.random.randint(50000,90000))
    elif d == 30:  # Data Science
        salaries.append(np.random.randint(90000,130000))
    elif d == 40:  # HR
        salaries.append(np.random.randint(45000,80000))
    elif d == 50:  # Marketing
        salaries.append(np.random.randint(55000,95000))

employees = pd.DataFrame({
    "emp_id": emp_ids,
    "emp_name": emp_names,
    "dept_id": dept_ids,
    "salary": salaries
})

employees.head()


,emp_id,emp_name,dept_id,salary
0,1,Emp1,20,72403
1,2,Emp2,50,73141
2,3,Emp3,30,104820
3,4,Emp4,30,96374
4,5,Emp5,10,76678


In [133]:
departments.head(10)

,dept_id,dept_name
0,10,Engineering
1,20,Sales
2,30,Data Science
3,40,HR
4,50,Marketing


In [134]:
joined_table = duckdb.query("""SELECT * FROM employees e left join departments d on e.dept_id = d.dept_id""").df()
joined_table.head(5)

,emp_id,emp_name,dept_id,salary,dept_id_1,dept_name
0,1,Emp1,20,72403,20,Sales
1,2,Emp2,50,73141,50,Marketing
2,3,Emp3,30,104820,30,Data Science
3,4,Emp4,30,96374,30,Data Science
4,5,Emp5,10,76678,10,Engineering


In [135]:
duckdb.register("joined_table", joined_table)

In [136]:
# Calculate employee count and average salary per department

joined_table_updated = duckdb.query("""select *, count(emp_name) over(partition by dept_id) as emp_count, avg(salary) over(partition by dept_id) as avg_dep_salary from joined_table""").df()

joined_table_updated.head(10)

,emp_id,emp_name,dept_id,salary,dept_id_1,dept_name,emp_count,avg_dep_salary
0,1,Emp1,20,72403,20,Sales,25,70239.2
1,17,Emp17,20,71834,20,Sales,25,70239.2
2,18,Emp18,20,68047,20,Sales,25,70239.2
3,19,Emp19,20,76105,20,Sales,25,70239.2
4,24,Emp24,20,73776,20,Sales,25,70239.2
5,25,Emp25,20,80080,20,Sales,25,70239.2
6,28,Emp28,20,77251,20,Sales,25,70239.2
7,37,Emp37,20,63545,20,Sales,25,70239.2
8,40,Emp40,20,57994,20,Sales,25,70239.2
9,42,Emp42,20,80303,20,Sales,25,70239.2


In [137]:
duckdb.register("joined_table_updated", joined_table_updated)

In [138]:
# Now we need to select all departments which have more than 10 employees

# duckdb.query("""
# with more_than_ten_emps as (select * from joined_table_updated where emp_count >=10)
# select *, rank() over(partition by ) as avg_salary_rank from more_than_ten_emps""")

more_than_10_emps = duckdb.query("""
select * from joined_table_updated where emp_count >=10""")
more_than_10_emps

┌────────┬──────────┬─────────┬────────┬───────────┬───────────┬───────────┬────────────────────┐
│ emp_id │ emp_name │ dept_id │ salary │ dept_id_1 │ dept_name │ emp_count │   avg_dep_salary   │
│ int64  │ varchar  │  int64  │ int64  │   int64   │  varchar  │   int64   │       double       │
├────────┼──────────┼─────────┼────────┼───────────┼───────────┼───────────┼────────────────────┤
│      1 │ Emp1     │      20 │  72403 │        20 │ Sales     │        25 │            70239.2 │
│     17 │ Emp17    │      20 │  71834 │        20 │ Sales     │        25 │            70239.2 │
│     18 │ Emp18    │      20 │  68047 │        20 │ Sales     │        25 │            70239.2 │
│     19 │ Emp19    │      20 │  76105 │        20 │ Sales     │        25 │            70239.2 │
│     24 │ Emp24    │      20 │  73776 │        20 │ Sales     │        25 │            70239.2 │
│     25 │ Emp25    │      20 │  80080 │        20 │ Sales     │        25 │            70239.2 │
│     28 │ Emp28    

In [139]:
duckdb.register("more_than_10_emps", more_than_10_emps)

In [141]:
duckdb.query("""select DISTINCT dept_id, avg_dep_salary from more_than_10_emps order by avg_dep_salary desc limit 3""").df()

,dept_id,avg_dep_salary
0,30,108031.117647
1,10,95095.029412
2,20,70239.200000


Ok, there is a much simpler method.

In [143]:
employees.head(2)

,emp_id,emp_name,dept_id,salary
0,1,Emp1,20,72403
1,2,Emp2,50,73141


In [142]:
departments.head(2)

,dept_id,dept_name
0,10,Engineering
1,20,Sales


In [147]:

simple_table = duckdb.query(
    """
    SELECT d.dept_id, d.dept_name, count(e.emp_id) as emp_count, avg(e.salary) as avg_salary
    FROM employees e left join departments d on e.dept_id = d.dept_id
    group by d.dept_id, d.dept_name having count(e.emp_id) > 10 order by avg_salary desc
    limit 3
    """
    ).df()
simple_table.head(5)

,dept_id,dept_name,emp_count,avg_salary
0,30,Data Science,17,108031.117647
1,10,Engineering,34,95095.029412
2,20,Sales,25,70239.200000


2. Calculate the first touch attribution channel for each user

In [104]:
import pandas as pd

touchpoints = pd.DataFrame({
    "user_id": [
        # User 1 (5 touches)
        1,1,1,1,1,
        # User 2 (4 touches)
        2,2,2,2,
        # User 3 (3 touches)
        3,3,3,
        # User 4 (2 touches)
        4,4,
        # User 5 (1 touch)
        5,
        # User 6 (6 touches)
        6,6,6,6,6,6,
        # User 7 (3 touches)
        7,7,7
    ],
    "event_time": [
        # User 1
        "2025-03-01 08:50:00",
        "2025-03-01 09:10:00",
        "2025-03-01 09:12:00",
        "2025-03-02 10:00:00",
        "2025-03-04 12:20:00",

        # User 2
        "2025-03-05 14:00:00",
        "2025-03-05 14:05:00",
        "2025-03-06 09:30:00",
        "2025-03-07 11:00:00",

        # User 3
        "2025-03-02 10:00:00",
        "2025-03-03 09:20:00",
        "2025-03-05 16:30:00",

        # User 4
        "2025-03-10 08:00:00",
        "2025-03-10 09:00:00",

        # User 5
        "2025-03-11 14:45:00",

        # User 6
        "2025-03-01 07:00:00",
        "2025-03-01 07:01:00",
        "2025-03-01 07:03:00",
        "2025-03-02 08:00:00",
        "2025-03-03 09:15:00",
        "2025-03-04 10:45:00",

        # User 7
        "2025-03-01 12:00:00",
        "2025-03-02 12:00:00",
        "2025-03-03 12:00:00"
    ],
    "channel": [
        # User 1
        "Organic","Facebook","Email","Paid Search","Instagram",

        # User 2
        "Paid Search","Instagram","Email","Facebook",

        # User 3
        "Organic","Email","Paid Search",

        # User 4
        "Referral","Organic",

        # User 5
        "Paid Search",

        # User 6
        "Facebook","Facebook","Email","Organic","Paid Search","Referral",

        # User 7
        "Email","Organic","Instagram"
    ]
})

touchpoints


,user_id,event_time,channel
0,1,2025-03-01 08:50:00,Organic
1,1,2025-03-01 09:10:00,Facebook
2,1,2025-03-01 09:12:00,Email
3,1,2025-03-02 10:00:00,Paid Search
4,1,2025-03-04 12:20:00,Instagram
5,2,2025-03-05 14:00:00,Paid Search
6,2,2025-03-05 14:05:00,Instagram
7,2,2025-03-06 09:30:00,Email
8,2,2025-03-07 11:00:00,Facebook
9,3,2025-03-02 10:00:00,Organic


In [106]:
touchpoints.head(5)

,user_id,event_time,channel
0,1,2025-03-01 08:50:00,Organic
1,1,2025-03-01 09:10:00,Facebook
2,1,2025-03-01 09:12:00,Email
3,1,2025-03-02 10:00:00,Paid Search
4,1,2025-03-04 12:20:00,Instagram


In [109]:
duckdb.query("""
with touchpoints_ordered as (select *, row_number() over(partition by user_id order by event_time) as event_time_rank from touchpoints
)
select * from touchpoints_ordered where event_time_rank = 1 order by user_id
""").df()

,user_id,event_time,channel,event_time_rank
0,1,2025-03-01 08:50:00,Organic,1
1,2,2025-03-05 14:00:00,Paid Search,1
2,3,2025-03-02 10:00:00,Organic,1
3,4,2025-03-10 08:00:00,Referral,1
4,5,2025-03-11 14:45:00,Paid Search,1
5,6,2025-03-01 07:00:00,Facebook,1
6,7,2025-03-01 12:00:00,Email,1


3. Dataset: Additional Purchases After Signup

In [111]:
users = pd.DataFrame({
    "user_id": [1,2,3],
    "signup_date": ["2025-03-01","2025-03-03","2025-03-04"]
})

purchases = pd.DataFrame({
    "purchase_id": [101,102,103,104,105],
    "user_id": [1,1,2,2,3],
    "purchase_date": ["2025-03-01","2025-03-10","2025-03-03","2025-03-06","2025-03-04"],
    "amount": [20,35,15,18,22]
})

In [112]:
users

,user_id,signup_date
0,1,2025-03-01
1,2,2025-03-03
2,3,2025-03-04


In [113]:
purchases

,purchase_id,user_id,purchase_date,amount
0,101,1,2025-03-01,20
1,102,1,2025-03-10,35
2,103,2,2025-03-03,15
3,104,2,2025-03-06,18
4,105,3,2025-03-04,22


In [115]:
duckdb.query(
"""
with joined_tables as (select * from purchases p left join users u on p.user_id = u.user_id)
select * from joined_tables where purchase_date > signup_date
""").df()

,purchase_id,user_id,purchase_date,amount,user_id_1,signup_date
0,102,1,2025-03-10,35,1,2025-03-01
1,104,2,2025-03-06,18,2,2025-03-03


4. Dataset: Average Swipes per User per Day

In [57]:
import pandas as pd
import numpy as np

np.random.seed(42)

users = [1, 2, 3]
days = pd.date_range("2025-03-01", periods=20, freq="D")

rows = []

for user in users:
    for day in days:
        num_swipes = np.random.randint(3, 7)  # 3 to 6 swipes per day
        times = pd.to_timedelta(
            np.random.randint(7*60, 22*60, num_swipes), unit="m"
        )
        for t in times:
            rows.append([user, day + t])

swipes = pd.DataFrame(rows, columns=["user_id", "swipe_time"])
swipes = swipes.sort_values(["user_id", "swipe_time"]).reset_index(drop=True)

swipes


,user_id,swipe_time
0,1,2025-03-01 08:11:00
1,1,2025-03-01 08:46:00
2,1,2025-03-01 11:30:00
3,1,2025-03-01 14:15:00
4,1,2025-03-01 21:20:00
...,...,...
274,3,2025-03-20 09:23:00
275,3,2025-03-20 12:45:00
276,3,2025-03-20 16:31:00
277,3,2025-03-20 17:23:00


In [64]:
duckdb.query("""
with swipes_updated as (select *, cast(swipe_time as DATE) as swipe_date, cast(swipe_time as TIME) as swipe_hour from swipes),
grouped_swipes as (select user_id, swipe_date, count(swipe_hour) as swipe_count from swipes_updated group by user_id, swipe_date)
select user_id, avg(swipe_count) from grouped_swipes group by user_id""").df()

,user_id,avg(swipe_count)
0,1,4.75
1,3,4.55
2,2,4.65


5. Write a query to retrieve the latest salary for each employee

In [88]:
import pandas as pd

salary_history = pd.DataFrame({
    "emp_id": [
        # Employee 1 (4 salary changes)
        1,1,1,1,
        # Employee 2 (3 salary changes)
        2,2,2,
        # Employee 3 (1 salary on record)
        3,
        # Employee 4 (5 salary changes)
        4,4,4,4,4,
        # Employee 5 (2 salary changes)
        5,5,
        # Employee 6 (3 salary changes)
        6,6,6
    ],
    "salary": [
        # Employee 1
        70000,72000,75000,80000,
        # Employee 2
        60000,63000,65000,
        # Employee 3
        90000,
        # Employee 4
        50000,52000,54000,56000,60000,
        # Employee 5
        88000,91000,
        # Employee 6
        45000,47000,50000
    ],
    "effective_date": [
        # Employee 1
        "2022-01-01","2023-01-01","2024-01-01","2025-01-01",
        # Employee 2
        "2023-06-01","2024-03-01","2025-02-01",
        # Employee 3
        "2024-07-01",
        # Employee 4
        "2021-02-01","2022-02-01","2023-02-01","2024-02-01","2025-02-01",
        # Employee 5
        "2023-11-15","2025-01-10",
        # Employee 6
        "2022-05-01","2023-05-01","2025-03-01"
    ]
})

salary_history


,emp_id,salary,effective_date
0,1,70000,2022-01-01
1,1,72000,2023-01-01
2,1,75000,2024-01-01
3,1,80000,2025-01-01
4,2,60000,2023-06-01
5,2,63000,2024-03-01
6,2,65000,2025-02-01
7,3,90000,2024-07-01
8,4,50000,2021-02-01
9,4,52000,2022-02-01


In [89]:
salary_history

,emp_id,salary,effective_date
0,1,70000,2022-01-01
1,1,72000,2023-01-01
2,1,75000,2024-01-01
3,1,80000,2025-01-01
4,2,60000,2023-06-01
5,2,63000,2024-03-01
6,2,65000,2025-02-01
7,3,90000,2024-07-01
8,4,50000,2021-02-01
9,4,52000,2022-02-01


In [102]:
duckdb.query("""
with salary_ordered as (select *, row_number() over(partition by emp_id order by effective_date desc) as salary_rank from salary_history
)
select * from salary_ordered where salary_rank = 1  order by emp_id
""").df()

,emp_id,salary,effective_date,salary_rank
0,1,80000,2025-01-01,1
1,2,65000,2025-02-01,1
2,3,90000,2024-07-01,1
3,4,60000,2025-02-01,1
4,5,91000,2025-01-10,1
5,6,50000,2025-03-01,1


6. Write a query to find the third purchase of every customer

In [83]:
import pandas as pd

orders = pd.DataFrame({
    "order_id": list(range(1, 31)),  # 30 orders total
    "customer_id": [
        # Customer 10: 10 orders
        10,10,10,10,10,10,10,10,10,10,
        # Customer 20: 8 orders
        20,20,20,20,20,20,20,20,
        # Customer 30: 6 orders
        30,30,30,30,30,30,
        # Customer 40: 6 orders
        40,40,40,40,40,40
    ],
    "order_date": [
        # Customer 10
        "2025-01-01","2025-01-03","2025-01-05","2025-01-06","2025-01-07",
        "2025-01-10","2025-01-15","2025-01-20","2025-01-25","2025-01-30",

        # Customer 20
        "2025-02-01","2025-02-02","2025-02-05","2025-02-09",
        "2025-02-11","2025-02-18","2025-02-20","2025-02-28",

        # Customer 30
        "2025-03-01","2025-03-03","2025-03-10","2025-03-12","2025-03-20","2025-03-25",

        # Customer 40
        "2025-04-01","2025-04-02","2025-04-05",
        "2025-04-10","2025-04-12","2025-04-15"
    ],
    "amount": [
        # Customer 10
        15,20,12,18,25,30,10,22,17,19,
        # Customer 20
        12,18,25,30,15,22,19,20,
        # Customer 30
        10,11,13,14,16,18,
        # Customer 40
        20,21,19,25,26,28
    ]
})


In [84]:
orders

,order_id,customer_id,order_date,amount
0,1,10,2025-01-01,15
1,2,10,2025-01-03,20
2,3,10,2025-01-05,12
3,4,10,2025-01-06,18
4,5,10,2025-01-07,25
5,6,10,2025-01-10,30
6,7,10,2025-01-15,10
7,8,10,2025-01-20,22
8,9,10,2025-01-25,17
9,10,10,2025-01-30,19


In [86]:
duckdb.query("""
with sub_query as(select *, dense_rank () over(partition by customer_id order by order_date) as order_num from orders order by customer_id, order_date)
select * from sub_query where order_num = 3
""").df()

,order_id,customer_id,order_date,amount,order_num
0,3,10,2025-01-05,12,3
1,13,20,2025-02-05,25,3
2,21,30,2025-03-10,13,3
3,27,40,2025-04-05,19,3


7. Write a query to return neighborhoods with no registered users

In [67]:
import pandas as pd

neighborhoods = pd.DataFrame({
    "neighborhood_id": list(range(1, 11)),
    "name": [
        "Uptown",
        "Downtown",
        "West Park",
        "Riverside",
        "Eastwood",
        "North Hills",
        "Southgate",
        "Lakeview",
        "Old Town",
        "Harbor District"
    ]
})

registered_users = pd.DataFrame({
    "user_id": list(range(1, 21)),
    "neighborhood_id": [
        1,1,1,   # Uptown (3 users)
        2,2,     # Downtown (2 users)
        3,       # West Park (1 user)
        5,5,5,5, # Eastwood (4 users)
        7,       # Southgate (1 user)
        8,8,     # Lakeview (2 users)
        10,10,10, # Harbor District (3 users)
        2,       # Downtown (extra)
        1,       # Uptown
        8,       # Lakeview
        7        # Southgate
    ]
})


In [68]:
neighborhoods

,neighborhood_id,name
0,1,Uptown
1,2,Downtown
2,3,West Park
3,4,Riverside
4,5,Eastwood
5,6,North Hills
6,7,Southgate
7,8,Lakeview
8,9,Old Town
9,10,Harbor District


In [69]:
registered_users

,user_id,neighborhood_id
0,1,1
1,2,1
2,3,1
3,4,2
4,5,2
5,6,3
6,7,5
7,8,5
8,9,5
9,10,5


In [71]:
duckdb.query("""
select * from neighborhoods n left join registered_users ru on
n.neighborhood_id = ru.neighborhood_id where user_id is NULL
""").df()

,neighborhood_id,name,user_id,neighborhood_id_1
0,9,Old Town,<NA>,<NA>
1,4,Riverside,<NA>,<NA>
2,6,North Hills,<NA>,<NA>


8. Capital One, a multinational bank, wants to identify their "whale users" from the credit card usage data. A "whale user" is a customer who makes a high number of transactions, amounting to a high monetary value. Create an SQL query that analyzes the customer credit card transaction data to list down the top 10 users who have the highest total transaction amounts in the last 30 days.(for this case, use more than 30 days).

In [149]:
import pandas as pd

# -----------------------------
# Transactions Table
# -----------------------------
transactions = pd.DataFrame({
    "transaction_id": [3221, 6732, 3982, 8744, 4637],
    "user_id": [765, 998, 473, 765, 473],
    "transaction_date": [
        "2023-09-12 00:00:00",
        "2023-09-15 00:00:00",
        "2023-09-18 00:00:00",
        "2023-09-12 00:00:00",
        "2023-09-19 00:00:00"
    ],
    "credit_card_id": [
        "1234567890123456",
        "2345678901234567",
        "3456789012345678",
        "1234567890123456",
        "3456789012345678"
    ],
    "transaction_amount": [750, 500, 1000, 1250, 3500]
})

# -----------------------------
# Users Table
# -----------------------------
users = pd.DataFrame({
    "user_id": [765, 998, 473],
    "first_name": ["John", "Jane", "Adrian"],
    "last_name": ["Doe", "Smith", "Johnson"]
})


In [150]:
transactions

,transaction_id,user_id,transaction_date,credit_card_id,transaction_amount
0,3221,765,2023-09-12 00:00:00,1234567890123456,750
1,6732,998,2023-09-15 00:00:00,2345678901234567,500
2,3982,473,2023-09-18 00:00:00,3456789012345678,1000
3,8744,765,2023-09-12 00:00:00,1234567890123456,1250
4,4637,473,2023-09-19 00:00:00,3456789012345678,3500


In [151]:
users

,user_id,first_name,last_name
0,765,John,Doe
1,998,Jane,Smith
2,473,Adrian,Johnson


In [186]:
whales_joined = duckdb.query("""
select t.user_id, t.transaction_amount, u.first_name, u.last_name from transactions t left join users u on
t.user_id = u.user_id where TODAY() - CAST(t.transaction_date as DATE) > 30
order by t.transaction_amount desc limit 10
""").df()

whales_joined

,user_id,transaction_amount,first_name,last_name
0,473,3500,Adrian,Johnson
1,765,1250,John,Doe
2,473,1000,Adrian,Johnson
3,765,750,John,Doe
4,998,500,Jane,Smith


In [194]:
# Remember to add up the transaction amounts per each whale

whales_joined_correct = duckdb.query("""
with joined_table as (select t.user_id, u.first_name, u.last_name, t.transaction_amount from transactions t left join users u on
t.user_id = u.user_id where TODAY() - CAST(t.transaction_date as DATE) > 30)
select user_id, first_name, last_name, sum(transaction_amount) as transaction_amount_sum from joined_table
group by user_id, first_name, last_name order by transaction_amount_sum desc limit 10
""").df()

whales_joined

,user_id,transaction_amount,first_name,last_name
0,473,3500,Adrian,Johnson
1,765,1250,John,Doe
2,473,1000,Adrian,Johnson
3,765,750,John,Doe
4,998,500,Jane,Smith


9. Top Department Salaries

Write a SQL query to find the top 3 highest earning employees within each department.

In [195]:
import pandas as pd

# Employee table
employees = pd.DataFrame({
    "employee_id": [1, 2, 3, 4, 5, 8, 10],
    "name": [
        "Emma Thompson",
        "Daniel Rodriguez",
        "Olivia Smith",
        "Noah Johnson",
        "Sophia Martinez",
        "William Davis",
        "James Anderson"
    ],
    "salary": [3800, 2230, 2000, 6800, 1750, 6800, 4000],
    "department_id": [1, 1, 1, 2, 1, 2, 1]
})

# Department table
departments = pd.DataFrame({
    "department_id": [1, 2],
    "department_name": ["Data Analytics", "Data Science"]
})

print("Employees Table:")
print(employees)

print("\nDepartments Table:")
print(departments)


Employees Table:
   employee_id              name  salary  department_id
0            1     Emma Thompson    3800              1
1            2  Daniel Rodriguez    2230              1
2            3      Olivia Smith    2000              1
3            4      Noah Johnson    6800              2
4            5   Sophia Martinez    1750              1
5            8     William Davis    6800              2
6           10    James Anderson    4000              1

Departments Table:
   department_id department_name
0              1  Data Analytics
1              2    Data Science


In [208]:
joined_tables = duckdb.query("""
with emp_and_deps as (select e.employee_id, e.name, e.salary, e.department_id, d.department_name
 from employees e left join departments d on e.department_id = d.department_id),
 emp_and_deps_window as (select *, dense_rank() over(partition by department_id order by salary desc) as salary_rank from emp_and_deps
)
select * from emp_and_deps_window where salary_rank < 4
""")

joined_tables

┌─────────────┬──────────────────┬────────┬───────────────┬─────────────────┬─────────────┐
│ employee_id │       name       │ salary │ department_id │ department_name │ salary_rank │
│    int64    │     varchar      │ int64  │     int64     │     varchar     │    int64    │
├─────────────┼──────────────────┼────────┼───────────────┼─────────────────┼─────────────┤
│          10 │ James Anderson   │   4000 │             1 │ Data Analytics  │           1 │
│           1 │ Emma Thompson    │   3800 │             1 │ Data Analytics  │           2 │
│           2 │ Daniel Rodriguez │   2230 │             1 │ Data Analytics  │           3 │
│           4 │ Noah Johnson     │   6800 │             2 │ Data Science    │           1 │
│           8 │ William Davis    │   6800 │             2 │ Data Science    │           1 │
└─────────────┴──────────────────┴────────┴───────────────┴─────────────────┴─────────────┘

10. Analyzing Monthly Average Spending on Credit Cards.

Calculate the monthly average transaction amounts for each card type from January 2022 to December 2022?

In [209]:
import pandas as pd

# Transactions table
transactions = pd.DataFrame({
    "transaction_id": [91872, 87263, 78142, 81005, 88364],
    "user_id": [458, 321, 156, 156, 983],
    "transaction_date": [
        "01/05/2022",
        "01/18/2022",
        "02/02/2022",
        "02/12/2022",
        "03/10/2022"
    ],
    "card_type": ["Visa", "Mastercard", "Visa", "Visa", "Mastercard"],
    "transaction_amount": ["$50", "$100", "$500", "$300", "$200"]
})

print(transactions)


   transaction_id  user_id transaction_date   card_type transaction_amount
0           91872      458       01/05/2022        Visa                $50
1           87263      321       01/18/2022  Mastercard               $100
2           78142      156       02/02/2022        Visa               $500
3           81005      156       02/12/2022        Visa               $300
4           88364      983       03/10/2022  Mastercard               $200


In [253]:
mnthly_avg_transactions = duckdb.query(
    """
    with date_slice as
    (select *, strftime(strptime(transaction_date, '%m/%d/%Y'), '%m') as trans_month from transactions where strptime(transaction_date, '%m/%d/%Y') between strptime('01/01/2022', '%m/%d/%Y') and strptime('12/31/2022', '%m/%d/%Y'))
    select DISTINCT trans_month, card_type, avg(CAST(SUBSTRING(transaction_amount, 2, LEN(transaction_amount) - 1) as INT)) over(partition by trans_month, card_type) as avg_transaction_amount from date_slice order by trans_month
    """
)
mnthly_avg_transactions

┌─────────────┬────────────┬────────────────────────┐
│ trans_month │ card_type  │ avg_transaction_amount │
│   varchar   │  varchar   │         double         │
├─────────────┼────────────┼────────────────────────┤
│ 01          │ Visa       │                   50.0 │
│ 01          │ Mastercard │                  100.0 │
│ 02          │ Visa       │                  400.0 │
│ 03          │ Mastercard │                  200.0 │
└─────────────┴────────────┴────────────────────────┘

Q11. Click-through rates

We want to calculate the CTR for each product. CTR is defined as the number of users who applied for a product after clicking on it divided by the number of total unique users who clicked on the product.

In [254]:
import pandas as pd

user_clicks = pd.DataFrame({
    "click_id": [8751, 9327, 6892, 7865, 5812],
    "user_id": [545, 298, 323, 299, 550],
    "product_id": [7412, 1258, 7412, 7412, 1258],
    "timestamp": [
        "07/08/2022 10:15:00",
        "07/09/2022 07:05:00",
        "07/09/2022 15:30:00",
        "07/10/2022 11:15:00",
        "07/11/2022 16:45:00"
    ]
})

user_clicks["timestamp"] = pd.to_datetime(user_clicks["timestamp"])

product_applications = pd.DataFrame({
    "application_id": [3874, 9285, 7903],
    "user_id": [545, 323, 299],
    "product_id": [7412, 7412, 7412],
    "application_time": [
        "07/08/2022 10:30:00",
        "07/09/2022 15:55:00",
        "07/11/2022 09:00:00"
    ]
})

product_applications["application_time"] = pd.to_datetime(product_applications["application_time"])


12. Find Customers from a Specific City

Filter out the customers who are based out of New York City.

In [255]:
import pandas as pd

customers = pd.DataFrame({
    "customer_id": [1190, 2325, 3782, 4911, 5761],
    "first_name": ["William", "Emma", "Olivia", "James", "Sophia"],
    "last_name": ["Smith", "Johnson", "Williams", "Brown", "Davis"],
    "city": ["New York", "Chicago", "New York", "Boston", "New York"],
    "sign_up_date": [
        "06/08/2020",
        "06/10/2020",
        "06/18/2020",
        "07/26/2020",
        "07/05/2020"
    ]
})

customers["sign_up_date"] = pd.to_datetime(customers["sign_up_date"])

In [256]:
customers

,customer_id,first_name,last_name,city,sign_up_date
0,1190,William,Smith,New York,2020-06-08
1,2325,Emma,Johnson,Chicago,2020-06-10
2,3782,Olivia,Williams,New York,2020-06-18
3,4911,James,Brown,Boston,2020-07-26
4,5761,Sophia,Davis,New York,2020-07-05


In [262]:
duckdb.query(
    """
    select customer_id, first_name, last_name from customers where city = 'New York'
    """)

┌─────────────┬────────────┬───────────┐
│ customer_id │ first_name │ last_name │
│    int64    │  varchar   │  varchar  │
├─────────────┼────────────┼───────────┤
│        1190 │ William    │ Smith     │
│        3782 │ Olivia     │ Williams  │
│        5761 │ Sophia     │ Davis     │
└─────────────┴────────────┴───────────┘

13.  Calculate Compound Monthly Interest Rates. Hard.

In [264]:
import pandas as pd

accounts = pd.DataFrame({
    "account_id": [101, 102, 103, 104, 105],
    "principal": [1000, 2000, 1500, 3000, 5000],
    "yearly_rate": [6, 7, 5.5, 6.5, 8],
    "monthly_deposit": [200, 300, 250, 600, 1000],
    "months": [6, 12, 12, 6, 12]
})

accounts

,account_id,principal,yearly_rate,monthly_deposit,months
0,101,1000,6.0,200,6
1,102,2000,7.0,300,12
2,103,1500,5.5,250,12
3,104,3000,6.5,600,6
4,105,5000,8.0,1000,12


In [267]:
duckdb.query(
    """
    with sub_query as ()
    select * from sub_query
    """
)

┌────────────┬────────────────────┐
│ account_id │   compount_value   │
│   int64    │       double       │
├────────────┼────────────────────┤
│        101 │ 1.0392310484577327 │
│        102 │ 1.0516335434979465 │
│        103 │  1.039307091672142 │
│        104 │  1.048791590293207 │
│        105 │ 1.0670947735489422 │
└────────────┴────────────────────┘

14 -24.

In [27]:
import pandas as pd

# ============================================================
# DATASET 1 — TRANSACTIONS
# ============================================================

transactions = pd.DataFrame({
    "transaction_id": [1,2,3,4,5,6,7],
    "customer_id":    [101,101,102,101,103,102,103],
    "amount":         [25.50,125.00,200.00,32.80,15.25,400.00,75.00],
    "merchant":       ["Starbucks","Amazon","Walmart","Starbucks","Starbucks","Apple","Walmart"],
    "txn_time": [
        "2022-01-02 08:15",
        "2022-01-05 13:45",
        "2022-01-06 19:10",
        "2022-01-07 08:05",
        "2022-01-07 09:00",
        "2022-01-09 14:25",
        "2022-01-10 17:45"
    ]
})

transactions["txn_time"] = pd.to_datetime(transactions["txn_time"])
print("transactions\n", transactions, "\n")


# ============================================================
# DATASET 2 — CUSTOMERS
# ============================================================

customers = pd.DataFrame({
    "customer_id": [101,102,103,104,105],
    "name":        ["Alice", "Bob", "Cara", "Dan", "Eva"],
    "state":       ["VA", "NY", "TX", "VA", "CA"],
    "signup_date": [
        "2021-10-01",
        "2020-12-15",
        "2022-02-20",
        "2021-05-03",
        "2020-01-10"
    ]
})

customers["signup_date"] = pd.to_datetime(customers["signup_date"])
print("customers\n", customers, "\n")


# ============================================================
# DATASET 3 — CREDIT PRODUCTS
# ============================================================

products = pd.DataFrame({
    "product_id":   [201,202,203,204,205],
    "product_type": ["Card","Card","Loan","Card","Loan"],
    "apr":          [16.5,21.0,8.25,17.9,5.0],
    "credit_limit": [2000,1000,None,1500,None]
})

print("products\n", products, "\n")


# ============================================================
# DATASET 4 — CLICKS
# ============================================================

clicks = pd.DataFrame({
    "click_id":     [1,2,3,4],
    "customer_id":  [101,102,101,105],
    "product_id":   [201,203,201,202],
    "click_time": [
        "2022-01-01",
        "2022-01-02",
        "2022-01-03",
        "2022-01-03"
    ]
})

clicks["click_time"] = pd.to_datetime(clicks["click_time"])
print("clicks\n", clicks, "\n")


# ============================================================
# DATASET 5 — APPLICATIONS
# ============================================================

applications = pd.DataFrame({
    "application_id": [99,100],
    "customer_id":    [101,102],
    "product_id":     [201,203],
    "app_time": [
        "2022-01-03",
        "2022-01-02"
    ]
})

applications["app_time"] = pd.to_datetime(applications["app_time"])
print("applications\n", applications, "\n")


# ============================================================
# DATASET 6 — RAW MERCHANT NAMES
# ============================================================

raw_merchants = pd.DataFrame({
    "raw_merchant": [
        "STARBUCKS #123",
        "Starbucks Store",
        "STARBUCKS-CA",
        "WALMART 88"
    ]
})

print("raw_merchants\n", raw_merchants, "\n")


transactions
    transaction_id  customer_id  amount   merchant            txn_time
0               1          101   25.50  Starbucks 2022-01-02 08:15:00
1               2          101  125.00     Amazon 2022-01-05 13:45:00
2               3          102  200.00    Walmart 2022-01-06 19:10:00
3               4          101   32.80  Starbucks 2022-01-07 08:05:00
4               5          103   15.25  Starbucks 2022-01-07 09:00:00
5               6          102  400.00      Apple 2022-01-09 14:25:00
6               7          103   75.00    Walmart 2022-01-10 17:45:00 

customers
    customer_id   name state signup_date
0          101  Alice    VA  2021-10-01
1          102    Bob    NY  2020-12-15
2          103   Cara    TX  2022-02-20
3          104    Dan    VA  2021-05-03
4          105    Eva    CA  2020-01-10 

products
    product_id product_type    apr  credit_limit
0         201         Card  16.50        2000.0
1         202         Card  21.00        1000.0
2         203    

EXERCISE 1 — Window Functions
Problem

For each customer, compute:

The total spending

The running total ordered by time

The rank of each purchase within each customer

Focus

Window functions, partition, order.

In [13]:
q_14 = duckdb.query("""
select * from transactions
""").df()

q_14

,transaction_id,customer_id,amount,merchant,txn_time
0,1,101,25.50,Starbucks,2022-01-02 08:15:00
1,2,101,125.00,Amazon,2022-01-05 13:45:00
2,3,102,200.00,Walmart,2022-01-06 19:10:00
3,4,101,32.80,Starbucks,2022-01-07 08:05:00
4,5,103,15.25,Starbucks,2022-01-07 09:00:00
5,6,102,400.00,Apple,2022-01-09 14:25:00
6,7,103,75.00,Walmart,2022-01-10 17:45:00


In [26]:
duckdb.query("""
select *,
sum(amount) over(partition by customer_id) as total_spending,
sum(amount) over(partition by customer_id order by txn_time ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_total_amount,
dense_rank() over(partition by customer_id order by amount desc) as purchase_rank  from transactions order by customer_id, txn_time
"""
).df()

,transaction_id,customer_id,amount,merchant,txn_time,total_spending,running_total_amount,purchase_rank
0,1,101,25.50,Starbucks,2022-01-02 08:15:00,183.30,25.50,3
1,2,101,125.00,Amazon,2022-01-05 13:45:00,183.30,150.50,1
2,4,101,32.80,Starbucks,2022-01-07 08:05:00,183.30,183.30,2
3,3,102,200.00,Walmart,2022-01-06 19:10:00,600.00,200.00,2
4,6,102,400.00,Apple,2022-01-09 14:25:00,600.00,600.00,1
5,5,103,15.25,Starbucks,2022-01-07 09:00:00,90.25,15.25,2
6,7,103,75.00,Walmart,2022-01-10 17:45:00,90.25,90.25,1
